<a href="https://colab.research.google.com/github/EvelynChrisyla/1.DataProcessingUsingSpark/blob/main/DataProcessingUsingSpark%5B1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark
from pyspark.sql.functions import*
from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=7bbff2d354fd71143a45a8c7e38948cd50c6aa2c64809275d724dc2156a304a6
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [5]:
# READ DATA
ramen_table = spark.read.option("inferSchema", "true").csv("MsRamen.csv", header = True)
customer_table = spark.read.option("inferSchema", "true").csv("MsCustomer.csv", header = True)
country_table= spark.read.option("inferSchema", "true").csv("MsCountry.csv", header = True)
brand_table= spark.read.option("inferSchema", "true").csv("MsBrand.csv", header = True)
Transactionheader_table= spark.read.option("inferSchema", "true").csv("TransactionHeader.csv", header = True)
Transactiondetail_table= spark.read.option("inferSchema", "true").csv("TransactionDetail.csv", header = True)

In [6]:
#convert the table into a table that can be read by Spark. because this table is still a dataframe
ramen_table.registerTempTable("MsRamen") #name the table
customer_table.registerTempTable("MsCustomer")
country_table.registerTempTable("MsCountry")
brand_table.registerTempTable("MsBrand")
Transactionheader_table.registerTempTable("TransactionHeader")
Transactiondetail_table.registerTempTable("TransactionDetail")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [22]:
#Checking
spark.sql("""
  SELECT
    RamenName
  From MsRamen
""").show()

+--------------------+
|           RamenName|
+--------------------+
|T's Restaurant Ta...|
|Noodles Spicy Hot...|
|Cup Noodles Chick...|
|GGE Ramen Snack T...|
|     Singapore Curry|
|Kimchi song Song ...|
|Spice Deli Tantan...|
|Nabeyaki Kitsune ...|
|Hokkaido Soy Sauc...|
|The Original Spic...|
|Creamy tom Yum Ku...|
|Yokohama Tonkotsu...|
|Mr. Bibim Stir-Fr...|
|Deka Buto Kimchi ...|
|Demae Ramen Bar N...|
|Mushroom Flavour ...|
|Mario Masala Noodles|
|   Tokyo Shoyu Ramen|
|          Mi Hai Cua|
|     Premium Gomtang|
+--------------------+
only showing top 20 rows



**1.  Show Top 5 Customer with the most transactions**


In [8]:
# Show Top 5 Customer with the most transactions

spark.sql("""
 SELECT
  CustomerName,
  COUNT(th.TransactionID) AS Totaltransaction
 FROM MsCustomer mc
 JOIN TransactionHeader th
 ON mc.CustomerID = th.CustomerID
 GROUP BY CustomerName
 ORDER BY TotalTRansaction DESC
 LIMIT 5
""").show()

+----------------+----------------+
|    CustomerName|Totaltransaction|
+----------------+----------------+
|    Elsa Hellens|              12|
|   Nevil Batisse|              11|
|  Karisa Tirrell|              11|
|Sergio Ungaretti|              11|
|  Fanya Glasscoe|              10|
+----------------+----------------+



In [9]:
customer_table.join(Transactionheader_table, "CustomerID").groupBy("CustomerName").agg(count("TransactionID").alias("TotalTransaction")).orderBy("TotalTransaction",ascending=False).limit(5).show()


+----------------+----------------+
|    CustomerName|TotalTransaction|
+----------------+----------------+
|    Elsa Hellens|              12|
|   Nevil Batisse|              11|
|  Karisa Tirrell|              11|
|Sergio Ungaretti|              11|
|  Fanya Glasscoe|              10|
+----------------+----------------+



**2. Show top 5 customer has spend the most in RamenShop with the quantity of transaction they bought is lower than the average quantity of all transaction.**

In [13]:
spark.sql("""
  SELECT CustomerName, TotalSpend
  FROM (
    SELECT mc.CustomerName, SUM(td.Quantity * mr.RamenPrice) AS TotalSpend
    FROM MsCustomer mc
    JOIN TransactionHeader th ON mc.CustomerID = th.CustomerID
    JOIN TransactionDetail td ON th.TransactionID = td.TransactionID
    JOIN MsRamen mr ON mr.RamenID = td.RamenID
    GROUP BY mc.CustomerName
    HAVING SUM(td.Quantity) < (SELECT AVG(Quantity) FROM TransactionDetail)
  ) AS subquery
  ORDER BY TotalSpend DESC
  LIMIT 5
""").show()


+------------------+----------+
|      CustomerName|TotalSpend|
+------------------+----------+
| Bobine Priestland|    200000|
|  Karlene Campbell|    190000|
|      Earvin Covey|    187000|
|Nikoletta Chetwynd|    184000|
|    Artur Gibbeson|    179000|
+------------------+----------+



**3. Show the Fequency of membership in RamenShop. Each Customer membership is determined by below condition:**




    Total Transaction  | Membership     
          <5               Bronze

          5-10             Silver

          >10               Gold    

In [21]:
spark.sql("""
    SELECT CustomerName,
    COUNT(th.TransactionID),
      CASE
        WHEN COUNT(th.TransactionID) <5 THEN "Bronze"
        WHEN COUNT(th.TransactionID) >10 THEN "Gold"
        ELSE "Silver"
      END AS Membership
    FROM MsCustomer mc
    JOIN TransactionHeader th
    ON mc.CustomerID = th.CustomerID
    GROUP BY  CustomerName
""").show()


+----------------+--------------------+----------+
|    CustomerName|count(TransactionID)|Membership|
+----------------+--------------------+----------+
|Lorene MacParlan|                   4|    Bronze|
|   Jacobo Zemler|                   4|    Bronze|
|   Arney Dunkley|                   5|    Silver|
|  Rhona Sutterby|                   6|    Silver|
|     Marve Dagon|                   6|    Silver|
|  Marilin Balfre|                   5|    Silver|
| Salomi Georgins|                   4|    Bronze|
|  Clint Faulkner|                   4|    Bronze|
| Kaitlyn McKaile|                   5|    Silver|
|    Stan Twydell|                   3|    Bronze|
|   Willey Mepham|                   4|    Bronze|
| Gabbey Silcocks|                   4|    Bronze|
|    Debee Kenson|                   6|    Silver|
|Ellette Gonsalvo|                   3|    Bronze|
|     Rhett Marco|                   2|    Bronze|
| Windham Houlson|                   3|    Bronze|
|     Lenci Brace|             